# Hello World of Optimization! 
Let's Make Some Casino Chips
## Our task...
We are in charge of chip production for a casino. 
- Management would like to make these new chips using the on-hand inventory of raw material. 
- Right now, we are asked to manufacture the **highest possible total value** in poker chips. 
- The denominations we can make are \\$1, \\$5, \\$10, \\$25, \\$100, \\$500, \$1000.
- Each denomination of chip requires a different amount of several raw materials.
- We are limited to the amount of material **on hand**. 

## Initial questions
- What are our decisions?
- What are our constraints? 
- What is the objective?
- What data do we need right now?

## The Model
A lot of programming (in Python) is *imperative* -- just providing sequential instructions to complete. But mathematical optimization (aka math programming) is *declarative*. The math programming model does not tell the Gurobi solver what to do specifically. Instead, the model tells the Gurobi solver what the solution must look like. Gurobi then finds the solution in its own way.

So math programming always starts with the creation of a new model. We then add to it the *declarations* about the final solution. 

Math optimization models take two forms:
- The **formulation**: An algebraic representation of the model (what we saw in our introduction!)
- The **code**: Writing the formulation in syntax to some software package.

In [22]:
#%pip install gurobipy

import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [23]:
model = gp.Model("Chip Manufacturing")

### The Variables

The model needs something to receive the solution values that the solver finds. It declares *variables* to hold those solution values.  The mathematical optimization model never explicitly sets these values. It just describes them and makes rules about the values they can hold. This model declares a variable for each type of chip.

Let $x_c$ be the number of chips made of type $c \in \{\$1, \$5, \$10, \$25, \$100, \$500, \$1000\}$.

First, let's create data for each chip type and its value.

In [24]:
chips = ["one", "five", "ten", "twenty-five", "one hundred", "five hundred", "thousand"]
data = pd.Series([1, 5, 10, 25, 100, 500, 1000], 
                  index=chips, 
                  name='value')
data

one                1
five               5
ten               10
twenty-five       25
one hundred      100
five hundred     500
thousand        1000
Name: value, dtype: int64

#### Anyone know another way to hardcode this another way?... using a `gurobipy` object?
**Multidict** function splits one dictionary into multiple, allowing for easier access to data while model building

In [25]:
chips, value = gp.multidict(
    {
        "one":             1,
        "five":            5,
        "ten":            10,
        "twenty-five":    25,
        "one hundred":   100,
        "five hundred":  500,
        "thousand":     1000
    }
)

In [26]:
print(chips)

['one', 'five', 'ten', 'twenty-five', 'one hundred', 'five hundred', 'thousand']


In [27]:
print(value)

{'one': 1, 'five': 5, 'ten': 10, 'twenty-five': 25, 'one hundred': 100, 'five hundred': 500, 'thousand': 1000}


Next, variables are added to the model -- one for each chip type. The main types of decision variables are
- CONTINUOUS
- INTEGER
- BINARY

But we'll explore more types later. 

#### What type of decision variable should we use in this case?
- Our decisions are the *number of chips* we should make for each chip

In [28]:
x = model.addVars(chips, vtype = GRB.INTEGER, name = "chips")
model.update()
x

{'one': <gurobi.Var chips[one]>,
 'five': <gurobi.Var chips[five]>,
 'ten': <gurobi.Var chips[ten]>,
 'twenty-five': <gurobi.Var chips[twenty-five]>,
 'one hundred': <gurobi.Var chips[one hundred]>,
 'five hundred': <gurobi.Var chips[five hundred]>,
 'thousand': <gurobi.Var chips[thousand]>}

### The Objective Function:

The math model must describe an objective also using algebra. When the model is solved, the value of the objective function will be the maximum or minimum possible while following the rules described in the constraints. 

This model will produce the highest possible value of chips. The objective function *multiplies the quantity of each chip produced times its value and that for all chips.
For example, the total value produced* by \$5 chips is $5*x_{five}$. 

So the total value is

\begin{equation*}
1*x_{one} + 5*x_{five} + 10*x_{ten} + ... + 1000*x_{thousand}
\end{equation*}

In [29]:
### Option 1 - Written term by term:
model.setObjective(x["one"] * value["one"] + 
                   x["five"] * value["five"] + 
                   x["ten"] * value["ten"] + 
                   x["twenty-five"] * value["twenty-five"] + 
                   x["one hundred"] * value["one hundred"] + 
                   x["five hundred"] * value["five hundred"] + 
                   x["thousand"] * value["thousand"], 
                    sense=GRB.MAXIMIZE)

Let $v_c$ be the value of chip $c$ and use a bit more math notation:
\begin{equation*}
  \text{Maximize} \space \sum_c v_c*x_c
\end{equation*}

In [30]:
### Option 2 - Here is another way to write the same thing:
model.setObjective(gp.quicksum(x[i] * value[i] for i in chips), sense=GRB.MAXIMIZE)

### Option 3-  The prod function is a handy function provided by the Gurobi API. 
model.setObjective(x.prod(value), sense=GRB.MAXIMIZE)

### The Constraints:

This casino has limited ingredients on hand to manufacture the chips needed. The model must make sure to only make chips for which the ingredients are available. 

Here is a list of ingredients and how much is needed for each type of chip:

In [31]:
ingredients, on_hand = gp.multidict(
    {
        "clay":      5000,
        "lead":      1500,
        "silver":     500,
        "gold":        50,
    }
)

In [32]:
print(ingredients)
print(on_hand)

['clay', 'lead', 'silver', 'gold']
{'clay': 5000, 'lead': 1500, 'silver': 500, 'gold': 50}


In [33]:
recipes = { 
        ("one",          "clay"):  18.0, ("one",          "lead"):  0.0, ("one",          "silver"):  0, ("one",          "gold"):  0, 
        ("five",         "clay"):  16.0, ("five",         "lead"):  1.0, ("five",         "silver"):  0, ("five",         "gold"):  0, 
        ("ten",          "clay"):  15.0, ("ten",          "lead"):  2.0, ("ten",          "silver"):  0, ("ten",          "gold"):  0, 
        ("twenty-five",  "clay"):  13.0, ("twenty-five",  "lead"):  4.5, ("twenty-five",  "silver"):  0, ("twenty-five",  "gold"):  0, 
        ("one hundred",  "clay"):  10.0, ("one hundred",  "lead"):  6.0, ("one hundred",  "silver"):  1, ("one hundred",  "gold"):  0, 
        ("five hundred", "clay"):  10.0, ("five hundred", "lead"):  8.5, ("five hundred", "silver"):  2, ("five hundred", "gold"):  0, 
        ("thousand",     "clay"):  10.0, ("thousand",     "lead"):  9.5, ("thousand",     "silver"):  0, ("thousand",     "gold"):  2, 
}

In [34]:
recipes['five', 'lead']

1.0

Let's introduce **constraints** to make sure we don't use more ingredients than we have on hand. Constraints are where the *rules* acting on decision variables are declared. 

For example, the amount of lead used for all chips made must be less than or equal to the total amount of lead on hand. 

\begin{equation*}
\text{total lead used} \le 1500
\end{equation*}

Let's write an expression for the total lead used using our decision variable for lead, $x_{lead}$.
\begin{align*}
\text{total lead used} = \space& lead_{one}* x_{one} + lead_{five}*x_{five} + ... + lead_{thousand}*x_{thousand} \\
&lead_{one}* x_{one} +  lead_{five}*x_{five} + ... + lead_{thousand}*x_{thousand} \le 1500
\end{align*}

Given we stored this information in the `recipes` dictionary, we let $r_{c,i}$ be the amount of *ingredient* $i$ used in making *chip* $c$.
\begin{equation*}
  r_{one, lead} * x_{one} + r_{five, lead}*x_{five} + ... + r_{thousand, lead}*x_{thousand} \le 1500
\end{equation*}


In [35]:
### Option 1 - A very explicit way to write these constraints:

# total clay <= clay on hand 
model.addConstr(x["one"]  * recipes["one", "clay"] + x["five"]  * recipes["five", "clay"] 
                + x["ten"]  * recipes["ten", "clay"] + x["twenty-five"]  * recipes["twenty-five", "clay"] 
                + x["one hundred"]  * recipes["one hundred", "clay"] + x["five hundred"]  * recipes["five hundred", "clay"] 
                + x["thousand"] * recipes["thousand", "clay"] <= on_hand["clay"], "clay limit")

# total lead <= lead on hand 
model.addConstr(x["one"]  * recipes["one", "lead"] + x["five"]  * recipes["five", "lead"] 
                + x["ten"]  * recipes["ten", "lead"] + x["twenty-five"]  * recipes["twenty-five", "lead"] 
                + x["one hundred"]  * recipes["one hundred", "lead"] + x["five hundred"]  * recipes["five hundred", "lead"] 
                + x["thousand"] * recipes["thousand", "lead"] <= on_hand["lead"], "lead limit")

# total gold <= gold on hand 
model.addConstr(x["one"]  * recipes["one", "gold"] + x["five"]  * recipes["five", "gold"] 
                + x["ten"]  * recipes["ten", "gold"] + x["twenty-five"]  * recipes["twenty-five", "gold"] 
                + x["one hundred"]  * recipes["one hundred", "gold"] + x["five hundred"]  * recipes["five hundred", "gold"] 
                + x["thousand"] * recipes["thousand", "gold"] <= on_hand["gold"], "gold limit")

# total silver <= silver on hand 
model.addConstr(x["one"]  * recipes["one", "silver"] + x["five"]  * recipes["five", "silver"] 
                + x["ten"]  * recipes["ten", "silver"] + x["twenty-five"]  * recipes["twenty-five", "silver"] 
                + x["one hundred"]  * recipes["one hundred", "silver"] + x["five hundred"]  * recipes["five hundred", "silver"] 
                + x["thousand"] * recipes["thousand", "silver"] <= on_hand["silver"], "silver limit")

<gurobi.Constr *Awaiting Model Update*>

We can also generalize the *quantity* of each ingredient with $q_i$. Then using a bit more mathematical notation:
\begin{equation*}
  \sum_{c}r_{c, i} * x_{c} \le q_i, \space \text{for all} \space i \space \text{in} \space \{\text{clay, lead, silver, gold}\}
\end{equation*}

Note: A short way to write "for all" is using the symbol $\forall$. Also, $\in$ means "in", or "an element of." We saw this notation in the intro.

Where the following can be found using recipes[('five', 'lead')]
\begin{equation*} r_{5, lead} \end{equation*} 

We can change the code so it looks for like this condensed notation and loop through each ingredient using `quicksum`: 

In [36]:
### Option 2 - Using quicksum
for ingredient in ingredients:
    model.addConstr(gp.quicksum(x[c] * recipes[c, ingredient] for c in chips) <= on_hand[ingredient], name="ingredients usage")

A little more compact, and makes it easier to store the constraints as an object:

In [37]:
### Option 2 - Using quicksum and storing constraint as an object 
balance_constraints = model.addConstrs((gp.quicksum(x[c] * recipes[c, i] for c in chips) <= on_hand[i] for i in ingredients), name="ingredients usage")

### The Solution:
It's as simple as one line of code to run the optimization, then we query the decision variables for their values (assuming the optimization completed successfully)

In [38]:
model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.6.0 24G231)

CPU model: Apple M3 Pro
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads

WLS license 2620953 - registered to Gurobi Optimization LLC
Optimize a model with 12 rows, 7 columns and 48 nonzeros
Model fingerprint: 0x499ca2fd
Variable types: 0 continuous, 7 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 5e+03]
Found heuristic solution: objective 302.0000000
Presolve removed 10 rows and 0 columns
Presolve time: 0.00s
Presolved: 2 rows, 7 columns, 13 nonzeros
Variable types: 0 continuous, 7 integer (0 binary)

Root relaxation: objective 9.944608e+04, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0

In [39]:
# use `VarName` and `X` to get the variables name and value, respectively:
for v in model.getVars():
    print(f"{v.VarName}: {v.X}")

chips[one]: 180.0
chips[five]: 0.0
chips[ten]: 0.0
chips[twenty-five]: 1.0
chips[one hundred]: -0.0
chips[five hundred]: 148.0
chips[thousand]: 25.0


Do you think another (and simpler) approach may work for this specific version of the problem?

## Congratulations!
You have just ran an optimization model!

# More Realistic Problems
Where we left off, a greedy approach would have worked to solve that model. But problems typically have much more complicated business rules to follow -- which highlights the strengths of mathematical optimzation. 

## Let's Create a function to make solving and reporting a bit more straightforward



In [40]:
%pip install gurobipy

import pandas as pd
import gurobipy as gp
from gurobipy import GRB

Note: you may need to restart the kernel to use updated packages.


In [41]:
def solve_and_print_solution(model,x,chips):
    # Optimize model
    model.setParam("outputflag", 0)
    model.optimize()
    
    # Check the model status
    if model.status == GRB.OPTIMAL:
        print("Optimal solution found")
        # Print the results
        for v in model.getVars():
            print(f"{v.VarName}: {v.X}")
        print(f"Objective value: {model.objVal}")
    else:
        print(f"Model status: {model.status}")
        sys.exit(1)


## Pick up where we left off
When setting up the problem the first time we used `multidict` to create parts of our model. This time, let's read data in from a csv. 

In [42]:
### This time, we read in data from csv
on_hand = pd.read_csv('data_files/ing_amounts.csv', index_col=['ingredient']).squeeze()
chips_data = pd.read_csv('data_files/chips_data.csv', index_col=['chips']).squeeze()
recipes = pd.read_csv('data_files/recipes.csv', index_col=['chips', 'ingredients']).squeeze()

ingredients = on_hand.index.to_list()
chips = chips_data.index.to_list()

## Original model

As a reminder:
- $x_c$ is the number of chips to make of denomination $c \in \{\$1, \$5, \$10, \$25, \$100, \$500, \$1000\}$.
- $v_c$ is the value associated with each chip.
- $q_i$ is the amount of ingredient $i \in \{\text{clay, lead, silver, gold}\}$ we have on hand.
- $r_{c,i}$ is the amount of **ingredient** $i$ used to make **chip** $c$. 

\begin{align*}
&\text{Maximize} \space \sum_c v_c*x_c \\
&\text{subject to:} \\
&\quad\sum_{c}r_{c, i} * x_{c} \le q_i, \space \text{for} \space i \in \{\text{clay, lead, silver, gold}\} \\
&\quad x_c \ge 0, \forall c \in \text{Chips}
\end{align*}

In [43]:
### Create a new model
model = gp.Model("Poker Chips")

### Decision variables
x = model.addVars(chips, vtype=GRB.INTEGER, name="chips")

### Ingredient constraint
ingredient_usage = model.addConstrs((gp.quicksum(x[c] * recipes[c, i] for c in chips) <= on_hand[i] for i in ingredients), name="ingredient_usage")

### Objective function
model.setObjective(gp.quicksum(x[i] * chips_data.value[i] for i in chips), sense=GRB.MAXIMIZE)

### Optimize 
solve_and_print_solution(model,x,chips)

Optimal solution found
chips[one]: 180.0
chips[five]: 0.0
chips[ten]: 0.0
chips[twenty-five]: 1.0
chips[one hundred]: -0.0
chips[five hundred]: 148.0
chips[thousand]: 25.0
Objective value: 99205.0


## Decision expressions
Some times it is helpful to store quantities of interest to help make code easier to read, reduce clutter, or get key values quickly.  

Suppose we have grouped chip types into two categories: low and high value. High value chips are \$100, \$500, and \$1000, with the rest being low value. 

Code the expressions below containing decision variables.

Let's define some sets. 
- $C = \{\$1, \$5, \$10, \$25, \$100\, \$500, \$1000\}$
- $H = \{\$100, \$500, \$1000\}$
- $L = C - H$

\begin{align*}
\text{total chips} &= \sum_{c \in C} x_c,   \\
\text{value of total chips} &= \sum_{c \in C} v_c*x_c   \\
\text{value of high chips} &= \sum_{h \in H} v_h * x_h    \\
\text{value of low chips} &= \sum_{l \in L} v_l*x_l    \\
\end{align*}

In [44]:
high_chips = ['one hundred','five hundred', 'thousand']
low_chips = [c for c in chips if c not in high_chips]

### While we use `quicksum` here, there are more compact ways to write these. 
total_chips  = gp.quicksum(x[c] for c in chips)
val_total_chips  = gp.quicksum(chips_data.value[c] * x[c] for c in chips)
val_high_chips = gp.quicksum(chips_data.value[c] * x[c] for c in high_chips)
val_low_chips  = gp.quicksum(chips_data.value[c] * x[c] for c in low_chips)

# Update model object to apply changes 
model.update()

In [45]:
### Test your expressions here
#val_low_chips.getValue() + val_high_chips.getValue()
total_chips.getValue()

354.0

## Changing our model

### New objective function
Instead of maximizing the total chip value, we are asked to maximize the total number of chips made. Write out and code this new objective

\begin{equation*}
\text{Maximize} \space \sum_{c \in C} x_c
\end{equation*}

In [46]:
### Set the objective to maximize the number of chips
model.setObjective(total_chips, GRB.MAXIMIZE)

# When you call optimize, the model is updated automatically, no need to call model.update()
solve_and_print_solution(model,x,chips)

Optimal solution found
chips[one]: 138.0
chips[five]: -0.0
chips[ten]: -0.0
chips[twenty-five]: 0.0
chips[one hundred]: 250.0
chips[five hundred]: -0.0
chips[thousand]: -0.0
Objective value: 388.0


Note that we didn't have to do anything else other than re-run `setObjective`. 

### Proportion of total chips

#### Formulate and code the following constraints
- The value produced by high value chips cannot exceed the 50% of the value produced by low value chips.
- Each of the of low value chips must be at least 15% of the total chips made and each of the high value chips cannot be more than 25%. 
- The \$25 chip is our most used; make sure that it has the maximum number among all chips.

\begin{equation*}
\sum_{h \in H} v_h*x_h \le 0.5*\sum_{l \in L} v_l*x_l
\end{equation*}

In [47]:
### The value produced by high value chips cannot exceed the 50% of the value produced by low value chips
value_balance = model.addConstr(val_high_chips <= .5*val_low_chips, name='value_balance')

solve_and_print_solution(model,x,chips)

Optimal solution found
chips[one]: 52.0
chips[five]: -0.0
chips[ten]: -0.0
chips[twenty-five]: 286.0
chips[one hundred]: 34.0
chips[five hundred]: -0.0
chips[thousand]: -0.0
Objective value: 372.0


We stored each constraint as an object, which makes it easier to interact with these later. Now, let's remove the `value_balance` constraint. 

In [48]:
model.remove(value_balance)

\begin{align*}
x_{l} \le& 0.20*\sum_{c \in C} x_c, \forall l\in L \\
x_{h} \ge& 0.05*\sum_{c \in C} x_c, \forall h\in H \\
\end{align*}

In [49]:
low_chip_ub = model.addConstrs((x[l] <= 0.20 * total_chips for l in low_chips), name='low_chip_ub')
high_chip_lb = model.addConstrs((x[h] >= 0.05 * total_chips for h in high_chips), name='high_chip_lb')

solve_and_print_solution(model,x,chips)

Optimal solution found
chips[one]: 74.0
chips[five]: 74.0
chips[ten]: 31.0
chips[twenty-five]: 1.0
chips[one hundred]: 152.0
chips[five hundred]: 19.0
chips[thousand]: 19.0
Objective value: 370.0


\begin{align*}
x_{ten} \ge x_{c}, \forall c \in C-\{\$10\}.
\end{align*}

In [50]:
### Use this set since this constraint doesn't need to apply to $25
C_minus25 = [cc for cc in chips if cc != 'twenty-five']

max_25 = model.addConstrs((x['twenty-five'] >= x[c] for c in [cc for cc in chips if cc != 'twenty-five']), name='max_25')

solve_and_print_solution(model,x,chips)

Optimal solution found
chips[one]: 72.0
chips[five]: 72.0
chips[ten]: 11.0
chips[twenty-five]: 72.0
chips[one hundred]: 72.0
chips[five hundred]: 45.0
chips[thousand]: 19.0
Objective value: 363.0


### Let's check in on our model and remove some constraints
As you code a model, you may want to make sure adding variables, constraints, and the objective all go as planned.

In [51]:
### Print the model object to get a quick summary
print(model)

### Writing a *.lp file is a great way to get a look at your model
model.write('our_model.lp')

<gurobi.Model MIP instance Poker Chips: 17 constrs, 7 vars, Parameter changes: LicenseID=2620953, OutputFlag=0>


The `Slack` of a constraint is the gap between the left-hand side and right-hand side values at the optimal solution. You can also think of this as how far the value is from it's bound, in our case upper bound. Printing this for the `ingredient_usage` constraints will show how much of each material is remaining. 

In [52]:
for i in ingredients:
    print(f"Remaining {i}: {ingredient_usage[i].Slack}")

Remaining clay: 8.0
Remaining lead: 4.0
Remaining gold: 12.0
Remaining silver: 338.0


In [53]:
## Lets remove the Proportion of Chips constraints
model.remove([low_chip_ub, high_chip_lb, max_25])
model.update()

## Look at which constraits remain in the model 
model.getConstrs()

[<gurobi.Constr ingredient_usage[clay]>,
 <gurobi.Constr ingredient_usage[lead]>,
 <gurobi.Constr ingredient_usage[gold]>,
 <gurobi.Constr ingredient_usage[silver]>]

### Meet minimum demand requirements
- View the chips data dataframe.
- Write constraints and code to meet a minimum demand for each chip value. 

In [54]:
### Print each chip here
chips_data

,value,demand,cost,fixed_cost
chips,,,,
one,1,30,0.05,10
five,5,40,0.10,20
ten,10,40,0.15,30
twenty-five,25,60,0.20,40
one hundred,100,20,0.25,50
five hundred,500,4,0.30,60
thousand,1000,2,0.35,70


Let $d_c$ be the demand for chip $c$.
\begin{equation*}
x_c \ge d_c, \forall c \in C
\end{equation*}

In [55]:
### Ensure the production meets the minimal chip demand
meet_demand = model.addConstrs((x[c] >= chips_data.demand[c] for c in chips), name="meet_demand")

solve_and_print_solution(model,x,chips)

Optimal solution found
chips[one]: 67.0
chips[five]: 40.0
chips[ten]: 40.0
chips[twenty-five]: 61.0
chips[one hundred]: 162.0
chips[five hundred]: 4.0
chips[thousand]: 2.0
Objective value: 376.0


### Cost of making chips:
- In the data frame above, we see there is a production cost per chip made. Write an expression for the total *variable cost* of chips made. 
- Make that the new objective to minimize total cost and solve. 
- Note that the demand constraints from before will still be active. 

Let $a_c$ be the variable cost of chip $c$. 
\begin{align*}
\text{total variable cost} = &\sum_c a_c*x_c  \\
\text{Minimize} &\sum_c a_c*x_c
\end{align*}

In [56]:
### Variable cost chip making cost
vCost_all_chips = sum(chips_data.cost[c] * x[c] for c in chips)

model.setObjective(vCost_all_chips, GRB.MINIMIZE)

solve_and_print_solution(model,x,chips)

Optimal solution found
chips[one]: 30.0
chips[five]: 40.0
chips[ten]: 40.0
chips[twenty-five]: 60.0
chips[one hundred]: 20.0
chips[five hundred]: 4.0
chips[thousand]: 2.0
Objective value: 30.4


### Fixed costs: More decision variables
Our production machine can only make chips of one value at a time. When we want to switch, it takes time to setup with new materials, which adds a fixed cost to each chip value. Given this, we have to decide whether or not we want to make each type of chip. If we decide to make at least one \$10 chip, then we incur a cost of 30. 

Let's go step-by-step to add in the fixed costs for each chip and minimize the total costs, while meeting the above demand.
- First define a new set of variables indexed by chip type and call them as `y`.
- Write a constraint that links the number of chips made with the new binary variable (**HINT:** use the cheat sheet). 
- Don't resolve yet. 


Let $f_c$ be the fixed cost of chip $c$ and $y_c = 1$ if chip $c$ is made, $0$ otherwise. 

If we decide to make a \$10 chip, then $x_{ten} > 0$ and we want to have the associated cost go from 0 to 30. 

So the constraint is
\begin{equation*}
x_c \le M*y_c, \forall c \in C
\end{equation*}

But what do we choose for $M$?


In [57]:
y = model.addVars(chips, vtype=GRB.BINARY, name="make")

### Link binary variable to chip production using `big-M` constraints
M = 500
model.addConstrs((x[c] <= M * y[c] for c in chips), name="link_x_y")
model.update()

### Fixed costs: Combining costs
- Write an expression that finds the total fixed costs using the new variable and the `fixed_cost` column from `chips_data`.
- Add that to the variable cost expression and set that as the new objective
- Solve!

\begin{align*}
&\text{total fixed cost} = \sum_c f_c*y_c  \\
\text{total cost} = \space&\text{total fixed cost}  + \text{total variable cost} = \sum_c f_c*y_c + a_c*x_c 
\end{align*}

In [58]:
fCost_all_chips = gp.quicksum(chips_data.fixed_cost[c] * y[c] for c in chips)
tCost_all_chips = vCost_all_chips + fCost_all_chips

model.setObjective(tCost_all_chips)

solve_and_print_solution(model,x,chips)

Optimal solution found
chips[one]: 30.0
chips[five]: 40.0
chips[ten]: 40.0
chips[twenty-five]: 60.0
chips[one hundred]: 20.0
chips[five hundred]: 4.0
chips[thousand]: 2.0
make[one]: 1.0
make[five]: 1.0
make[ten]: 1.0
make[twenty-five]: 1.0
make[one hundred]: 1.0
make[five hundred]: 1.0
make[thousand]: 1.0
Objective value: 310.4


Let's remove the demand constraint and see what happens. 

In [59]:
model.remove(meet_demand)
model.setObjective(total_chips, sense=GRB.MAXIMIZE)
solve_and_print_solution(model,x,chips)

Optimal solution found
chips[one]: 138.0
chips[five]: -0.0
chips[ten]: -0.0
chips[twenty-five]: 0.0
chips[one hundred]: 250.0
chips[five hundred]: -0.0
chips[thousand]: -0.0
make[one]: 1.0
make[five]: 1.0
make[ten]: 1.0
make[twenty-five]: 1.0
make[one hundred]: 1.0
make[five hundred]: 1.0
make[thousand]: 1.0
Objective value: 388.0


### Logical constraints with binary variables
Now that we have binary decision variables that show which chip types are made, we can model logical relationships between them. Model the following statements and write `gurobipy` code. 

We won't solve a model with these. 
- We **can** make either \$500 **or** \$1000 chips, and **possibly both** (at least one).
- We **must** make either \$500 **or** \$1000 chips, but **not both** (exactly one).
- We make between 2 and 3 types of low value. 

- We **can** make either \$500 **or** \$1000 chips, and **possibly both** (at least one).
\begin{equation*}
y_{five\space hundred} + y_{thousand} \le 1 
\end{equation*}

- We **must** make either \$500 **or** \$1000 chips, but **not both** (exactly one).
\begin{equation*}
y_{five\space hundred} + y_{thousand} = 1 
\end{equation*}

- We make between 2 and 3 types of low value. 
\begin{equation*}
2 \le \sum_l y_l \le 3
\end{equation*}

In [60]:
# We can make either $500 or $1000 chips, but not both (at most one)
model.addConstr(y['five hundred'] + y['thousand'] <= 1, name="at_most_one_high_value")

# We must make either $500 or $1000 chips, but not both (exactly one)
model.addConstr(y['five hundred'] + y['thousand'] == 1, name="exactly_one_high_value")

# We make between 2 and 3 types of low value and exactly one of the high value chips
model.addConstr(gp.quicksum(y[l] for l in low_chips) >= 2, name="min_low_value_types")
model.addConstr(gp.quicksum(y[l] for l in low_chips) <= 3, name="max_low_value_types")

### Another shortcut using the gurobipy API
model.addConstr(gp.quicksum(y[l] for l in low_chips) == [2,3], name="max_low_value_types")


<gurobi.Constr *Awaiting Model Update*>

### Conditional statements
- If we make \$1 chips, then we **must** make \$5 chips.
- If we make \$1 chips, then we **must** make \$5 **and** \$10 chips.
- If we make \$1 chips, then we **must** make \$5 **or** \$10 chips (or both). 

- If we make \$1 chips, then we **must** make \$5 chips.
\begin{align*}
y_{one} &\le y_{five} \quad\quad\quad\space
\end{align*}

- If we make \$1 chips, then we **must** make \$5 **and** \$10 chips.

\begin{align*}
y_{one} &\le y_{five}, \\
y_{one} &\le y_{ten} \\
&\text{or} \\
2*y_{one} &\le y_{five} + y_{ten}
\end{align*}

- If we make \$1 chips, then we **must** make \$5 **or** \$10 chips (or both). 

\begin{align*}
\quad y_{one} &\le y_{five} + y_{ten}
\end{align*}


In [61]:
#### Logistical constraint candidates:

### If we make $1 chips, then we must make $5 chips
model.addConstr(y['one'] <= y['five'], name="one_implies_five")

### If we make $1 chips, then we must make $5 and $10 chips
model.addConstr(y['one'] <= y['five'], name="one_implies_five")
model.addConstr(y['one'] <= y['ten'], name="one_implies_ten")

model.addConstr(2*y['one'] <= y['five'] + y['ten'], name="one_implies_ten")

### If we make $1 chips, then we must make $5 or $10 chips
model.addConstr(y['one'] <= y['five'] + y['ten'], name="one_implies_ten")

<gurobi.Constr *Awaiting Model Update*>

## Multi-objective optimization
- We are asked to maximize the total chip value made while minimizing costs. 
- Math optimization cannot **simultaneously** work on two objectives -- there's always a tradeoff. 
- Two types on multi-objective: hierarchical, and weighted (blended).
- After talking more about how we want to prioritize the objectives, we want to:
    - First minimize the **total costs** (both fixed and variable) to meet demand. 
    - Then maximize the total value of chips produced while not decreasing chip value by 10%
- Also include the following constraints we already discussed:
    - Ingredient limits
    - Meet demand
    - Make the most \$25 chips

### DIY hierarchical multi-objective optimzation
- Solve the a model that maximizes the total chip value made.
- Add a new constraint that uses this objective value as a new bound on total chip value. Let $z$ be this value.
\begin{equation*}
\sum_c v_c*x_c \ge 0.9*z
\end{equation*}
- Set the new objective to minimize total cost, as before. 
- Solve!

In [62]:
model = gp.Model("Poker Chips")

M = 500

### Decision variables
x = model.addVars(chips, vtype=GRB.INTEGER, name="chips")
y = model.addVars(chips, vtype=GRB.BINARY, name="make")

### Other expressions
val_total_chips  = gp.quicksum(chips_data.value[c] * x[c] for c in chips)
vCost_all_chips = sum(chips_data.cost[c] * x[c] for c in chips)
fCost_all_chips = gp.quicksum(chips_data.fixed_cost[c] * y[c] for c in chips)
tCost_all_chips = vCost_all_chips + fCost_all_chips

### Constraints
ingredient_usage = model.addConstrs((gp.quicksum(x[c] * recipes[c, i] for c in chips) <= on_hand[i] for i in ingredients), name="ingredient_usage")
max_25 = model.addConstrs((x['twenty-five'] >= x[c] for c in [cc for cc in chips if cc != 'twenty-five']), name='max_25')
meet_demand = model.addConstrs((x[c] >= chips_data.demand[c] for c in chips), name="meet_demand")
link_x_y = model.addConstrs((x[c] <= M * y[c] for c in chips), name="link_x_y")

### First Objective
model.setObjective(val_total_chips, sense=GRB.MAXIMIZE)

solve_and_print_solution(model,x,chips)

Optimal solution found
chips[one]: 72.0
chips[five]: 40.0
chips[ten]: 40.0
chips[twenty-five]: 72.0
chips[one hundred]: 21.0
chips[five hundred]: 72.0
chips[thousand]: 25.0
make[one]: 1.0
make[five]: 1.0
make[ten]: 1.0
make[twenty-five]: 1.0
make[one hundred]: 1.0
make[five hundred]: 1.0
make[thousand]: 1.0
Objective value: 65572.0


In [ ]:
z = model.ObjVal
print(z)
multi_obj = model.addConstr(val_total_chips >= 0.9*z)
model.setObjective(tCost_all_chips)
solve_and_print_solution(model,x,chips)

In [ ]:
print(0.9*z)
val_total_chips.getValue()

#### Homework!
Use the **cheat sheet** to do this using the `gurobipy` multi-objective functionality.

In [ ]:
model.dispose()